In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import gc

# 分布確認に使う
#import pandas_profiling as pdp
# 可視化
import matplotlib.pyplot as plt
# 前処理、特徴量作成 - sklearnを使う
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# モデリング・精度と評価指標
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
#LGBM
import lightgbm as lgb

# NOTE matplotでの日本語文字化けを解消
#pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline

# NOTE matplotでの日本語文字化けを解消
#pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline

In [ ]:
import pandas as pd
pd.options.display

train = pd.read_csv("train.csv", index_col=0) # 学習用データ
test = pd.read_csv("test.csv", index_col=0) # 学習用データ   # 評価用データ
sample_submit = pd.read_csv("sample_submission.csv", index_col=0, header=None) # 応募用サンプルファイル

In [ ]:
train.head()

print("データ形状：")
print(train.shape)

print("データ数：")
print(len(train))

print("データのコラム数")
print(len(train.columns))

print("データ型一覧")
train.info()

In [ ]:
train.head(20)

コラム数は全部で20個。ダミー化できるものはダミー化、そうでないものは分布数の多いもののみを取り出すなどの工夫をして
ドメイン知識を使った特徴量エンジニアリングをする。

In [ ]:
print("データ型一覧")
train.info()

tree_dbh, health, borocode, boro_ct, cb_num, st_senate, st_assem, cncldistで
ベースラインを作成する。その他本来ならint型にするべき値もベースラインに用いる

tree_dbh - 木の円周。量的変数
health - 木の健康状態、目的変数
borocode - ニューヨークの行政区分。質的変数
boro_ct - ニューヨーク市の行政区分（ボロー）の名称。質的変数
cb_num - コミュニティボード番号。質的変数
st_senate - 州上院地区番号。質的変数
st_assem - 州議会地区番号。質的変数
cncldist - 市議会の地区番号。質的変数

→使えるのはtree_dbhだけ。
次はstr・object型からint・float型に変換するべきものを探す

steward → 木の管理者数 
stewardはダミー化する。

tree_dbh,stewardの他にもまずは簡単にダミー化できる変数5つを使ってベースを作成する

In [ ]:
print("データのコラム数")
print(train.columns)

In [ ]:
print("データのコラム数")
print(test.columns)

In [ ]:
#目的変数の分布を確認
counts = train["health"].value_counts()
counts

In [ ]:
counts = counts.sort_index()

In [ ]:
counts.plot.bar("health")

目的変数は1(good)であることが殆どだが、0や2(normal, bad)が約20%。

In [ ]:
#目的変数の分布を確認
counts = train["curb_loc"].value_counts()
print(counts)
counts = counts.sort_index()
counts.plot.bar("curb_loc")

In [ ]:
#目的変数の分布を確認
counts = train["steward"].value_counts()
print(counts)
counts = counts.sort_index()
counts.plot.bar("steward")

In [ ]:
#目的変数の分布を確認
counts = train["guards"].value_counts()
print(counts)

counts = counts.sort_index()
counts.plot.bar("guards")

In [ ]:
#目的変数の分布を確認
counts = train["sidewalk"].value_counts()
print(counts)

counts = counts.sort_index()
counts.plot.bar("sidewalk")

In [ ]:
#目的変数の分布を確認
counts = train["user_type"].value_counts()
print(counts)

counts = counts.sort_index()
counts.plot.bar("user_type")

In [ ]:
train["tree_dbh"].hist(bins=20)

In [ ]:
counts

→ ポアソン分布に近い。指数化すればより有効な値になるかも？

In [ ]:
#目的変数の分布を確認
counts = train["problems"].value_counts()
print(counts)

counts = counts.sort_index()
counts.plot.bar("problems")

In [ ]:
#目的変数の分布を確認
prob_counts = train["problems"].value_counts()
print(prob_counts)

#counts = counts.sort_index()
prob_counts[:10]

In [ ]:
spc_counts = train["spc_common"].value_counts()
print(spc_counts)

#counts = counts.sort_index()
spc_counts.plot.bar("spc_common")

In [ ]:
# 木の種類 - 最初の10をダミー化する
counts[:15].sum()

In [ ]:
counts = train["spc_latin"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("spc_latin")

In [ ]:
# 木の種類 - 最初の10をダミー化する
counts[:10]

In [ ]:
#地域（Neighborhood Tabulation Area）の略称
counts = train["nta"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("nta")

In [ ]:
#地域（Neighborhood Tabulation Area）の略称
counts = train["nta_name"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("nta_name")

In [ ]:
# vill, beach, side, hill, park, water, bay, gardenなど木に影響しそうな名称を抽出するか？？
counts[:30]

In [ ]:
#ニューヨーク市の行政区分（ボロー）のコード
#ボロコードは非常に有益そう。量的変数にする
counts = train["borocode"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("borocode")

In [ ]:
counts = train["boro_ct"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("boro_ct")

In [ ]:
#　ニューヨーク市の行政区分（ボロー）の名称
#　ボロー名称は有効そう。ダミー変数にする

counts = train["boroname"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("boroname")

In [ ]:
#　郵便番号に関連する都市または地区
#　最初の5-10個をダミー化するか??
# 頻度を量的変数にする、などより有効なダミー化も検討中

counts = train["zip_city"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("zip_city")

In [ ]:
#　ニューヨーク市の行政区分（ボロー）の名称
#　ボロー名称は有効そう。ダミー変数にする

counts = train["cb_num"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("cb_num")

In [ ]:
#　ニューヨーク市の行政区分（ボロー）の名称
#　ボロー名称は有効そう。ダミー変数にする

counts = train["st_senate"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("st_senate")

In [ ]:
#　ニューヨーク市の行政区分（ボロー）の名称
#　ボロー名称は有効そう。ダミー変数にする

counts = train["st_assem"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("st_assem")

In [ ]:
#　ニューヨーク市の行政区分（ボロー）の名称
#　ボロー名称は有効そう。ダミー変数にする

counts = train["cncldist"].value_counts()
print(counts)

#counts = counts.sort_index()
counts.plot.bar("cncldist")

In [ ]:
len(counts)

→最初の10個の問題をダミー化する

In [ ]:
probs = prob_counts.index
problist = probs[:20]
problist 

In [ ]:
# NOTE　not_problist - 10のproblem以外の変数を除外するためのリスト
not_problist = probs[20:]
not_problist

In [ ]:
#木の種類15種をダミー化するためのリスト
spcs = spc_counts.index
spclist = spcs[:20]
spclist

In [ ]:
# NOTE　not_problist - 10のproblem以外の変数を除外するためのリスト
not_spcslist = spcs[20:]
not_spcslist

# 前処理

# ベースライン作成に用いる変数
tree_dbh
curb_loc
steward
guards
sidewalk
user_type

In [ ]:
x_list = ["tree_dbh","curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
y_list = ["health"]
dummy_list = ["curb_loc","steward","guards","sidewalk","user_type","borocode"]

In [ ]:
train.head(5)

In [ ]:
train.head(5)

## 木の健康状態（目的変数）
0:Fair（普通） 1:Good（良い） 2:Poor（悪い）
0:良い、1:普通、2:悪い、という値に置き換える

In [ ]:
train['health'] = train['health'].map({0:1, 1:0, 2:2})
train

In [ ]:
#　説明変数+目的変数とバリデーション設計
X_train, y_train = train[x_list], train[y_list]
X_train

In [ ]:
y_train

In [ ]:
X_train.isnull().sum()

→　欠損値は'NULL'というカテゴリに置き換え、欠損であるという情報=質的変数の1つとして処理する

In [ ]:
train.fillna('NULL', inplace=True)

In [ ]:
column_means = train.groupby("steward")["health"].mean("health")
column_means

In [ ]:
test["steward"]

In [ ]:
X_train.fillna('NULL', inplace=True)

In [ ]:
X_train.head(20)

In [ ]:
# problemsをbool変換 → 問題があるか否かのダミー変数にする
# train['bool_problems'] = train['problems'].apply(lambda x: 0 if x=='NULL' else 1)

In [ ]:
#　木の直径とhealthの関係
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

plt.figure(figsize=(10, 6))
sns.boxplot(x='health', y='tree_dbh', data=train)
plt.title('木の円周・木の健康状態の箱ひげ図')
plt.show()

→ badな木は若干円周が大きい。育ちすぎ等の原因があるかもしれない

# 特徴量エンジニアリング

今回の特徴量エンジニアリング

x_list = ["tree_dbh", "curb_loc","steward","guards","sidewalk","user_type"]
y_list = ["health"]
dummy_list = ["curb_loc","steward","guards","sidewalk","user_type"]

- ["curb_loc","steward","guards","sidewalk","user_type"]をダミー化する
- "problems"をダミー化する。その際最初の10行だけを残す


# 質的変数をターゲットエンコーディング
まず各コラムの平均値を求める

In [ ]:
# 一挙にターゲットエンコーディング
cols = ["curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
for col in cols:
    column_means = train.groupby(col)["health"].mean("health")
    column_means
    
    colname = col + "_tg_enc"
    
    X_train[colname] = train[col].map(column_means)

In [ ]:
X_train

In [ ]:
# problems, spc_commonをドロップアウト
drop_col = ["curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]

X_train = X_train.drop(drop_col , axis=1)
X_train

In [ ]:
# 可視化ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

colormap = plt.cm.RdBu

In [ ]:
corr = X_train.corr()

In [ ]:
sns.heatmap(corr, square=True, 
            cmap=colormap, 
            linecolor='white', annot=False,
            vmin=1.0, vmax=-1.0)

In [ ]:
X_train["problems_tg_enc"].hist(bins = 20)

In [ ]:
# NOTE not_problistに含まれている変数を除外する
# NOTE not_spcslistに服あれている変数を除外する
"""
num = 17 + len(problist)
X_train_02 = X_train.iloc[:, 0:num]
X_train_02
"""
"""
X_train = pd.get_dummies(X_train, columns = ["problems"], drop_first=False)
X_train

for column in not_problist:
    
    colname = "problems_" + str(column)
    X_train = X_train.drop(columns=colname, axis=1)
"""

In [ ]:
"""
X_train = pd.get_dummies(X_train, columns = ["spc_common"], drop_first=False)
X_train

for spc in not_spcslist:
    
    colname = "spc_common_" + str(spc)
    X_train = X_train.drop(columns=colname, axis=1)
    
X_train
"""

## 質的変数を頻度を表す量的変数に変更する

In [ ]:
# 特徴量：ProblemsFreq - Ticket頻度をチケットを表す量的変数とする
#X_train.loc[:, 'Problems_Freq'] = X_train.groupby(['problems'])['problems'].transform('count')
#X_train

In [ ]:
# 特徴量：ProblemsFreq - Ticket頻度をチケットを表す量的変数とする
#X_train.loc[:, 'Spcs_Freq'] = X_train.groupby(['spc_common'])['spc_common'].transform('count')
#X_train

ここまで実行

In [ ]:
# problems, spc_commonをドロップアウト
#drop_col = ["problems","spc_common"]
#X_train = X_train.drop(drop_col , axis=1)

# 特徴量選択
学習に使う特徴量を半分に絞る


In [ ]:
"""
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor

# estimatorとしてGBDTを使用。特徴量を20個選択
selector = RFE(GradientBoostingRegressor(n_estimators=100, random_state=10), n_features_to_select=20)
selector.fit(X_train, y_train)
mask = selector.get_support()
#print(X.feature_names)
print(mask)

# 選択した特徴量の列のみ取得
X_selected = selector.transform(X_train)
print("X.shape={}, X_selected.shape={}".format(X_train.shape, X_selected.shape))

list = []
not_selected = []
columns = X_train.columns

for i in range(0, len(mask)):
    
    value = mask[i]
    
    if (value == True):
        list.append(columns[i])
    else:
        not_selected.append(columns[i])

print("選択された20の特徴量：")
print(list)

print("選択されなかった特徴量：")
print(not_selected)
"""

# 標準化

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_std = pd.DataFrame(std_scaler.transform(X_train), columns=X_train.columns)
X_train_std.describe()

In [ ]:
X_train_std.head(10)

# X_train_stdを次元圧縮

In [ ]:
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.manifold import TSNE

In [ ]:
pca = decomposition.PCA()
X_pca = pca.fit_transform(X_train_std)
X_pca

In [ ]:
X_pca.shape

In [ ]:
X_tsne_nc = TSNE(learning_rate = 100).fit_transform(X_train_std)
print(X_tsne_nc[:5, :5])
print("--- X_tsne_nopca ---")

In [ ]:
X_tsne = TSNE(learning_rate = 100).fit_transform(X_pca)
print(X_tsne[:5, :5])
print("--- X_tsne ---")

In [ ]:
plt.figure(figsize = (12, 12))

plt.subplot(221)
plt.scatter(X_train_std["tree_dbh"], X_train_std["problems_tg_enc"],color = "blue", alpha = 0.25)

plt.subplot(222)
plt.scatter(X_tsne_nc[:,0], X_tsne_nc[:,1],color = "blue", alpha = 0.25)

plt.subplot(223)
plt.scatter(X_pca[:,0], X_pca[:,1],color = "blue", alpha = 0.25)

plt.subplot(224)
plt.scatter(X_tsne[:,0], X_tsne[:,1],color = "blue", alpha = 0.25)

plt.show()

In [ ]:
次元圧縮は pca→t-SNEの順に行うのが効果的

In [ ]:
# ホールドアウト検証 - 学習用・テスト用の分割を1通り決める
X_tr, X_va, y_tr, y_va = train_test_split(X_tsne, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=123)

print("学習用・訓練用データの形状：")
print(X_tr.shape)
print(y_tr.shape)
print(X_va.shape)
print(y_va.shape)

# モデル最適化

In [ ]:
import optuna

# LGBMのパラメータ
params_base = {"boosting_type":"gbdt",
          "objective":"multiclass",
          "metric":"multi_logloss",
          "num_class":"3",
          "learning_rate":0.01,
          "n_estimators":1000,
          "random_state":123,
          "importance_type":"gain",
          "early_stopping_round":100,
          "verbose":10
          }

def objective(trial):
    from sklearn.metrics import f1_score

# チューニングで探索する最適パラメータ
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf":trial.suggest_int("min_data_in_leaf", 5,200),
        "min_sum_hessian_in_leaf":trial.suggest_float("min_sum_hessian_in_leaf", 0.00001, 0.01, log = True),
        "feature_fraction":trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction":trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1":trial.suggest_float("lambda_l1", 0.01, 10.0, log = True),
        "lambda_l2":trial.suggest_float("lambda_l2", 0.01, 10.0, log = True)
    }
    
    #tuningにbaseの値を加える
    params_tuning.update(params_base)
    
    #モデル学習(ベイズ最適化)
    list_metrics=[]
    
    # ホールドアウト検証 - 学習用・テスト用の分割を1通り決める
    X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=123)

        
    model = lgb.LGBMClassifier(**params_tuning)
        
    model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_va, y_va)], 
                    #early_stopping_rounds=100, 
                    #verbose=0
                    )
    y_va_pred = model.predict(X_va)
    
    #検証精度を求める
    metric_va = f1_score(y_va, y_va_pred, average='macro')
    #metric_va = accuracy_score(y_va, y_va_pred)
    return metric_va

# 探索を実行
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=100)

In [ ]:
# 探索で得られた結果を確認
trial = study.best_trial
print("最も高いF1スコア")
print(trial.value)

print("最も高い精度となるパラメータ：")
print(trial.params)

最も高い精度：
0.7880910683012259
最も高い精度となるパラメータ：
{'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hessian_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'lambda_l1': 0.18591711878786357, 'lambda_l2': 8.755734725056497}

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
# LGBMのモデル
model = lgb.LGBMClassifier(**params_best)
model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_va, y_va)])

In [ ]:
# AUC値に加え精度を算出する
y_tr_pred = model.predict(X_tr)
y_va_pred = model.predict(X_va)

metric_tr = accuracy_score(y_tr, y_tr_pred)
metric_va = accuracy_score(y_va, y_va_pred)
print("モデル精度:")
print("学習精度")
print(metric_tr)
print("検証精度")
print(metric_va)

In [ ]:
from sklearn.metrics import f1_score

valid_predictions = model.predict(X_va)
valid_f1 = f1_score(y_va, valid_predictions, average='macro')

print("マクロf1スコア：")
print(valid_f1)

micro_f1 = f1_score(y_va, valid_predictions, average='micro')

print("ミクロf1スコア：")
print(micro_f1)

In [ ]:
from sklearn.metrics import balanced_accuracy_score

bas_score = balanced_accuracy_score(y_va, valid_predictions)

print("比重付き精度：")
print(bas_score)

In [ ]:
# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_va_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

→ようやく改善が見られた。1, 2のラベルが正確に分類されるようにする。

In [ ]:
# 寄与率算出
# feature importanceを表示
# importanceを表示する
importance = pd.DataFrame(model.feature_importances_, index=X_tr.columns, columns=['importance'])
impo02 = importance.sort_values(by="importance")
display(impo02)

In [ ]:
lgb.plot_importance(model)

problems_TrunkOther	0.000000
problems_StonesTrunkOther	2.406300
problems_BranchOther	10.268320
steward_3or4	13.906020
problems_RootOtherTrunkOtherBranchOther	20.130170
steward_4orMore	22.130930
curb_loc_OnCurb	26.089900
problems_StonesRootOther	29.473130
steward_NULL	31.843760
sidewalk_NoDamage	37.184240
problems_StonesBranchOther	38.574340
curb_loc_OffsetFromCurb	38.776410
problems_BranchLights	39.326430
guards_Unsure	40.097710
problems_RootOther	49.345920
guards_Harmful	54.356240

problemsは殆ど寄与していないものもある
problems_TrunkOther	0.000000
problems_StonesTrunkOther	2.406300
problems_BranchOther	10.268320
problems_RootOtherTrunkOtherBranchOther	20.130170
problems_StonesRootOther	29.473130


In [ ]:
X_train.columns

In [ ]:
ここまで実行

# ロジスティック回帰による予測


In [ ]:
# 
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(random_state=0)
model_lr.fit(X_tr, y_tr)

y_pred = model_lr.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score

valid_predictions = model_lr.predict(X_va)
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

In [ ]:
# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

In [ ]:
# 近傍中心法で学習
from sklearn.neighbors import NearestCentroid

model_nc = NearestCentroid()
model_nc.fit(X_tr, y_tr)

y_pred = model_nc.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score

valid_predictions = model_nc.predict(X_va)
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

In [ ]:
# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# ガウシアンナイーブベイズによる予測

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_gnb = GaussianNB()
model_gnb.fit(X_tr, y_tr)

y_pred = model_gnb.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score

valid_predictions = model_gnb.predict(X_va)
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# ランダムフォレスト法による予測

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr)

y_pred = model_rf.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score

valid_predictions = model_rf.predict(X_va)
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

reg = LazyClassifier(ignore_warnings=True, random_state=1121, verbose=False,predictions=True)
models, predictions = reg.fit(X_tr, X_va, y_tr, y_va) 

print("モデルの精度・評価指標：")
display(models)
print("テストデータの予測値：")
display(predictions)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

model_pac = PassiveAggressiveClassifier()
model_pac.fit(X_tr, y_tr)

y_pred = model_pac.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# ExtraTreeClassifierを学習

In [ ]:
from sklearn.tree import ExtraTreeClassifier

model_etc = ExtraTreeClassifier()
model_etc.fit(X_tr, y_tr)

y_pred = model_etc.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# DecisionTreeClassifierを学習

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_dtc = DecisionTreeClassifier()
model_dtc.fit(X_tr, y_tr)

y_pred = model_dtc.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# QuadraticDiscriminantAnalysisを学習

In [ ]:
#　QuadraticDiscriminantAnalysisを学習
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model_qda = QuadraticDiscriminantAnalysis()
model_qda.fit(X_tr, y_tr)

y_pred = model_qda.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# LinearDiscriminantAnalysis

In [ ]:
#　QuadraticDiscriminantAnalysisを学習
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_lda = LinearDiscriminantAnalysis()
model_lda.fit(X_train_std, y_train)

y_pred = model_lda.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# BaggingClassifierを学習

In [ ]:
#　QuadraticDiscriminantAnalysisを学習
from sklearn.ensemble import BaggingClassifier

model_bc = BaggingClassifier()
model_bc.fit(X_tr, y_tr)

y_pred = model_bc.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

In [ ]:
# Perceptronを学習

In [ ]:
from sklearn.linear_model import Perceptron

model_pcpt = BaggingClassifier()
model_pcpt.fit(X_tr, y_tr)

y_pred = model_pcpt.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model_etc = ExtraTreesClassifier()
model_etc.fit(X_tr, y_tr)

y_pred = model_etc.predict(X_va)
y_pred

In [ ]:
bas_score = balanced_accuracy_score(y_va, y_pred)

print("比重付き精度：")
print(bas_score)

from sklearn.metrics import f1_score
valid_f1 = f1_score(y_va, y_pred, average='macro')

print("マクロf1スコア：")
print(valid_f1)

# 混合行列 - ラベルの正誤の分類数をまとめる
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_pred, y_va)
print("混合行列：")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2"])
disp.plot()
plt.show()

# 投稿用データを作成

In [ ]:
# 説明変数・目的変数
X_test = test[x_list]
X_test

# 欠損値補完
X_test.fillna('NULL', inplace=True)
X_test

In [ ]:
# 一挙にターゲットエンコーディング
cols = ["curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
for col in cols:
    column_means = train.groupby(col)["health"].mean("health")
    column_means
    
    colname = col + "_tg_enc"
    
    X_test[colname] = X_test[col].map(column_means)

In [ ]:
# problems, spc_commonをドロップアウト
drop_col = ["curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
X_test = X_test.drop(drop_col , axis=1)
X_test

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
std_scaler.fit(X_test)
X_test_std = pd.DataFrame(std_scaler.transform(X_test), columns=X_train.columns)
X_test_std
# X_train_std.describe()

In [ ]:
# 予測
y_test_pred = model_pac.predict(X_test_std)
y_test_pred

In [ ]:
# NOTE 予測結果を1次元ベクトルにする
y_test_pred = np.squeeze(y_test_pred)
y_test_pred.shape
y_test_pred

In [ ]:
# NOTE 予測結果をpdにして処理
df_submit = pd.DataFrame(data=y_test_pred, columns=['y_pred'])
df_submit

In [ ]:
df_submit.value_counts()

In [ ]:
df_submit['y_pred'] = df_submit['y_pred'].map({0:1, 1:0, 2:2})
df_submit

In [ ]:
df_submit.value_counts()

In [ ]:
y_test_pred = df_submit["y_pred"].to_numpy()

In [ ]:
sample_submit[1] = y_test_pred
sample_submit

In [ ]:
#木の健康状態を0・1を置き換える
#train['health'] = train['health'].map({0:1, 1:0, 2:2})
#train

sample_submit.to_csv('submit_PassiveAggressiveClassifier_01.csv', header=None)